In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [3]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [4]:
model_name = "NousResearch/llama-2-7b-chat-hf"

In [8]:
question = "what are the sales in august 2023"
prompt = f"""[INST]<<SYS>>
Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english.
For example if user gives "top  orders in march 2020", you should only return march 2020.
<</SYS>>

{question}
[/INST]
[END]"""

In [9]:
import json
BASE_MODEL = model_name

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = "auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt = json.dumps(prompt)
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
model.eval()
response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

"[INST]<<SYS>>\nAssume you are a system that detects the month and year in the given secentance, user will give only provide the input in english.\nFor example if user gives \"top  orders in march 2020\", you should only return march 2020.\n<</SYS>>\n\nwhat are the sales in august 2023\n[/INST]\n[END]"

The user has provided the input "what are the sales in august 2023". Based on the given input, I can detect that the month is "august" and the year is "2023". Therefore, the output is "august 2023".
